In [1]:
import pandas as pd

In [2]:
import psycopg2
from psycopg2.extras import execute_values

In [3]:
sattelites_types = {
    'name': 'string',
    'operator': 'string',
    'country_of_operator': 'string',
    'contractor': 'string',
    'country_of_contractor': 'string',
    'purpose': 'string',
    'users': 'string',
    'date_of_launch': 'string' ,
    'class_of_orbit': 'string',
    'orbit_period': 'float64',
    'launch_mass': 'float64',
    'Date of Launch': 'string',
    'expected_lifetime': 'float64',
    'launch_site': 'string',
    'rocket_name': 'string'
}
sattelites_df = pd.read_excel("../data/standardized_data/sattelites.xlsx",dtype=sattelites_types)

In [4]:
launches_df= pd.read_excel("../data/standardized_data/launches.xlsx")

In [5]:
rocket_types = {
    'name': 'string',
    'company': 'string',
    'status': 'string',
    'liftoff_thrust Thrust': 'float64',
    'payload_leo': 'float64',
    'stages': 'object',
    'height': 'float64',
    'price': 'float64',
}
rockets_df= pd.read_excel("../data/extracted_data/rockets.xlsx",dtype=rocket_types)

In [6]:
satellite_counts = sattelites_df.groupby(["date_of_launch", "launch_site","rocket_name"]).size().reset_index(name="satellite_count")

In [7]:
dff = satellite_counts[satellite_counts["satellite_count"]>1].sort_values(by="satellite_count", ascending=False)

In [81]:
# 2. Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="space_dw",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5433"
)
cur = conn.cursor()
conn.rollback()
cur.execute("SELECT 1;")


In [82]:
# 3. Prepare Dimension Tables
launch_site_dim = launches_df[['space_port', 'country']].drop_duplicates()
rocket_dim = rockets_df[['name', 'company', 'status', 'liftoff_thrust', 'payload_leo', 'stages', 'height', 'price']].drop_duplicates()
rocket_dim = rocket_dim.where(pd.notnull(rocket_dim), None)
launch_dim = launches_df[['status', 'provider', 'mission','launch_pad']].drop_duplicates()
satellite_dim = sattelites_df[['name', 'country_of_operator', 'operator', 'users', 'purpose', 'class_of_orbit', 'orbit_period', 'launch_mass', 'expected_lifetime', 'contractor', 'country_of_contractor']].drop_duplicates()
satellite_dim = satellite_dim.where(pd.notnull(satellite_dim), None)

In [83]:
date_dim = pd.DataFrame()
date_dim['full_date'] = pd.to_datetime(launches_df['date']).drop_duplicates()
date_dim['year'] = date_dim['full_date'].dt.year
date_dim['month'] = date_dim['full_date'].dt.month
date_dim['day'] = date_dim['full_date'].dt.day

In [84]:
# 4. Insert Dimensions (Bulk Insert)
def bulk_insert_dim(table, df, returning_key):    
    records = [tuple(x) for x in df.to_numpy()]
    columns = ','.join(df.columns)
    placeholders = ','.join(['%s'] * len(df.columns))
    
    insert_sql = f"INSERT INTO {table} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING RETURNING {returning_key};"
    keys = []
    for row in records:
        cur.execute(insert_sql, row)
        res = cur.fetchone()
        if res:
            keys.append(res[0])
    return keys

In [85]:
satellite_dim['country_of_contractor'] = satellite_dim['country_of_contractor'].apply(
    lambda x: x if isinstance(x, list) else [x]
)
satellite_dim['country_of_operator'] = satellite_dim['country_of_operator'].apply(
    lambda x: x if isinstance(x, list) else [x]
)

In [86]:
# Insert dimensions
bulk_insert_dim('dim_launch_site', launch_site_dim, 'launch_site_key')
bulk_insert_dim('dim_rocket', rocket_dim, 'rocket_key')
bulk_insert_dim('dim_satellite', satellite_dim, 'satellite_key')
bulk_insert_dim('dim_launch_site', launch_site_dim.head(1), 'launch_site_key')
bulk_insert_dim('dim_date', date_dim, 'date_key')

[23821,
 23822,
 23823,
 23824,
 23825,
 23826,
 23827,
 23828,
 23829,
 23830,
 23831,
 23832,
 23833,
 23834,
 23835,
 23836,
 23837,
 23838,
 23839,
 23840,
 23841,
 23842,
 23843,
 23844,
 23845,
 23846,
 23847,
 23848,
 23849,
 23850,
 23851,
 23852,
 23853,
 23854,
 23855,
 23856,
 23857,
 23858,
 23859,
 23860,
 23861,
 23862,
 23863,
 23864,
 23865,
 23866,
 23867,
 23868,
 23869,
 23870,
 23871,
 23872,
 23873,
 23874,
 23875,
 23876,
 23877,
 23878,
 23879,
 23880,
 23881,
 23882,
 23883,
 23884,
 23885,
 23886,
 23887,
 23888,
 23889,
 23890,
 23891,
 23892,
 23893,
 23894,
 23895,
 23896,
 23897,
 23898,
 23899,
 23900,
 23901,
 23902,
 23903,
 23904,
 23905,
 23906,
 23907,
 23908,
 23909,
 23910,
 23911,
 23912,
 23913,
 23914,
 23915,
 23916,
 23917,
 23918,
 23919,
 23920,
 23921,
 23922,
 23923,
 23924,
 23925,
 23926,
 23927,
 23928,
 23929,
 23930,
 23931,
 23932,
 23933,
 23934,
 23935,
 23936,
 23937,
 23938,
 23939,
 23940,
 23941,
 23942,
 23943,
 23944,
 23945,


In [87]:
conn.commit()

In [88]:
# Merge satellites with launches (using rocket, launch_site, date)
sattelites_df['date_of_launch'] = pd.to_datetime(sattelites_df['date_of_launch'])
merged_df = sattelites_df.merge(
    launches_df,
    left_on=['date_of_launch', 'rocket_name', 'launch_site'],
    right_on=['date', 'rocket', 'space_port'],
    how='inner'
)

In [89]:
merged_df

,name,country_of_operator,operator,users,purpose,class_of_orbit,orbit_period,launch_mass,date_of_launch,expected_lifetime,...,rocket_name,date,status,provider,mission,launch_site_y,launch_pad,space_port,country,rocket
0,AAC AIS-Sat1 (Kelpie 1),['United Kingdom'],AAC Clyde Space,Commercial,Earth Observation,LEO,95.00,4.0,2023-01-03,NaN,...,Falcon 9,2023-01-03,Success,SpaceX,Transporter 6 (Dedicated SSO Rideshare),"Space Launch Complex 40 | Cape Canaveral, FL, USA",Space Launch Complex 40,Cape Canaveral,USA,Falcon 9
1,ABS-2,['Multinational'],Asia Broadcast Satellite Ltd.,Commercial,Communications,GEO,1436.03,6330.0,2014-02-06,15.0,...,Ariane 5 ECA,2014-02-06,Success,Arianespace,ABS-2 & Athena-Fidus,"Ariane Launch Area 3 | Guiana Space Centre, Fr...",Ariane Launch Area 3,Guiana Space Centre,French Guiana,Ariane 5 ECA
2,ABS-2A,['Multinational'],Asia Broadcast Satellite Ltd.,Commercial,Communications,GEO,1436.10,1800.0,2016-06-15,15.0,...,Falcon 9,2016-06-15,Success,SpaceX,Eutelsat 117 West B & ABS-2A,"Space Launch Complex 40 | Cape Canaveral, FL, USA",Space Launch Complex 40,Cape Canaveral,USA,Falcon 9
3,ADLER-2,['Austria'],Austrian Space Forum (OEWF)/Spire,Civil,Earth Science,LEO,95.00,11.5,2023-04-15,NaN,...,Falcon 9,2023-04-15,Success,SpaceX,Transporter 7 (Dedicated SSO Rideshare),"Space Launch Complex 4E | Vandenberg SFB, CA, USA",Space Launch Complex 4E,Vandenberg SFB,USA,Falcon 9
4,USA 223,['USA'],National Reconnaissance Office (NRO),Military,Earth Observation,GEO,1436.00,5000.0,2010-11-21,NaN,...,Delta IV Heavy,2010-11-21,Success,United Launch Alliance,NROL-32 (Mentor) (USA-223),"Space Launch Complex 37B | Cape Canaveral, FL,...",Space Launch Complex 37B,Cape Canaveral,USA,Delta IV Heavy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3338,OHS-6,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3339,OHS-7,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3340,OHS-8,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11
3341,OVS-3,['China'],Zhuhai Orbita Control Engineering Co. Ltd.,Commercial,Earth Observation,LEO,94.60,55.0,2019-09-19,NaN,...,Long March 11,2019-09-19,Success,China Aerospace Science and Technology Corpora...,Zhuhai-1-03,Launch Area 95A | Jiuquan Satellite Launch Cen...,Launch Area 95A,Jiuquan Satellite Launch Center,China,Long March 11


In [90]:
fact_records = []

for _, row in merged_df.iterrows():
    # ----------------------------
    # Dim Satellite
    cur.execute("SELECT satellite_key FROM dim_satellite WHERE name = %s", (row['name'],))
    res = cur.fetchone()
    if res:
        satellite_key = res[0]
    else:
        cur.execute("INSERT INTO dim_satellite (name) VALUES (%s) RETURNING satellite_key", (row['name'],))
        satellite_key = cur.fetchone()[0]

    # ----------------------------
    # Dim Rocket
    cur.execute("SELECT rocket_key FROM dim_rocket WHERE name = %s", (row['rocket_name'],))
    res = cur.fetchone()
    if res:
        rocket_key = res[0]
    else:
        cur.execute("INSERT INTO dim_rocket (name) VALUES (%s) RETURNING rocket_key", (row['rocket_name'],))
        rocket_key = cur.fetchone()[0]

    # ----------------------------
    # Dim Launch Site
    cur.execute("SELECT launch_site_key FROM dim_launch_site WHERE space_port = %s", (row['space_port'],))
    res = cur.fetchone()
    if res:
        launch_site_key = res[0]
    else:
        cur.execute("INSERT INTO dim_launch_site (space_port, country) VALUES (%s, %s) RETURNING launch_site_key",
                    (row['space_port'], row.get('country', 'Unknown')))
        launch_site_key = cur.fetchone()[0]

    # ----------------------------
    # Dim Date
    cur.execute("SELECT date_key FROM dim_date WHERE full_date = %s", (row['date_of_launch'],))
    res = cur.fetchone()
    if res:
        date_key = res[0]
    else:
        cur.execute("INSERT INTO dim_date (full_date) VALUES (%s) RETURNING date_key", (row['date_of_launch'],))
        date_key = cur.fetchone()[0]

    # ----------------------------
    # Dim Launch
    cur.execute("""
        SELECT launch_key FROM dim_launch 
        WHERE status = %s AND provider = %s AND mission = %s
    """, (row['status'], row['provider'], row['mission']))
    res = cur.fetchone()
    if res:
        launch_key = res[0]
    else:
        cur.execute("""
            INSERT INTO dim_launch (status, provider, mission) 
            VALUES (%s, %s, %s) RETURNING launch_key
        """, (row['status'], row['provider'], row['mission']))
        launch_key = cur.fetchone()[0]

    # ----------------------------
    # Append to fact table records
    fact_records.append((satellite_key, rocket_key, launch_site_key, date_key, launch_key))

# ----------------------------
# Insertion dans la fact table (optionnel)
for rec in fact_records:
    cur.execute("""
        INSERT INTO fact_launch (satellite_key, rocket_key, launch_site_key, date_key, launch_key)
        VALUES (%s, %s, %s, %s, %s)
    """, rec)

# Commit changes
conn.commit()


In [91]:
# Bulk insert into fact_satellite_launch
execute_values(cur,"INSERT INTO fact_satellite_launch (satellite_key, rocket_key, launch_site_key, date_key, launch_key)VALUES %s", fact_records)

In [92]:
conn.commit()
cur.close()
conn.close()